In [1]:
import os

import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from netCDF4 import Dataset  # pylint:disable=no-name-in-module
from shapely.geometry import Point

In [2]:
nc = Dataset(
    os.path.join(
        os.environ["HOME"],
        "Documents",
        "eocis",
        "landice_dash_test",
        "data_files",
        "EOCIS-AIS-L3C-SEC-MULTIMISSION-5KM-5YEAR-MEANS-201705-202205-fv1.nc",
    )
)

In [3]:
for k, v in nc.variables.items():
    print(k, "\n", v)
    print()

sec 
 <class 'netCDF4._netCDF4.Variable'>
float32 sec(time_period, ny, nx)
    long_name: surface elevation change
    units: m/yr
    source: multi-mission radar altimetry
    grid_mapping: grid_projection
unlimited dimensions: 
current shape = (1, 968, 1128)
filling on, default _FillValue of 9.969209968386869e+36 used

sec_uncertainty 
 <class 'netCDF4._netCDF4.Variable'>
float32 sec_uncertainty(time_period, ny, nx)
    long_name: uncertainty in surface elevation change
    units: m/yr
    grid_mapping: grid_projection
unlimited dimensions: 
current shape = (1, 968, 1128)
filling on, default _FillValue of 9.969209968386869e+36 used

x 
 <class 'netCDF4._netCDF4.Variable'>
float32 x(nx)
    long_name: Cartesian x-coordinate - easting, of centre of each grid cell
    units: meters
    standard_name: projection_x_coordinate
    min_val: -2817500.0
    binsize: 5000.0
unlimited dimensions: 
current shape = (1128,)
filling on, default _FillValue of 9.969209968386869e+36 used

y 
 <class '

In [4]:
x_values: np.ndarray = nc["x"][:].data
y_values: np.ndarray = nc["y"][:].data
lat: np.ndarray = nc["lat"][:].data
lon: np.ndarray = nc["lon"][:].data
surf_type: np.ndarray = nc["surface_type"][:].data
sec: np.ndarray = np.squeeze(nc["sec"][:].data)

x_coords, y_coords = np.meshgrid(x_values, y_values, indexing="xy")

print(x_values.shape, y_values.shape)
print(x_coords.shape, y_coords.shape, sec.shape, surf_type.shape)

(1128,) (968,)
(968, 1128) (968, 1128) (968, 1128) (968, 1128)


In [5]:
coords_arr = [Point(x, y) for x, y in zip(x_coords.flatten(), y_coords.flatten())]
sec_arr = sec.flatten()
surf_type_arr = surf_type.flatten()

print(len(coords_arr), sec_arr.shape, surf_type_arr.shape)

1091904 (1091904,) (1091904,)


In [6]:
np.unique(surf_type)

array([0, 1, 2, 3, 4], dtype=int8)

In [7]:
my_data = gpd.GeoDataFrame(
    data={
        "SEC": sec_arr,
        "surface type": surf_type_arr,
        "geometry": coords_arr,
    },
    crs="epsg:3031",
)
my_data.head()

,SEC,surface type,geometry
0,NaN,0,POINT (-2817500.000 -2417500.000)
1,NaN,0,POINT (-2812500.000 -2417500.000)
2,NaN,0,POINT (-2807500.000 -2417500.000)
3,NaN,0,POINT (-2802500.000 -2417500.000)
4,NaN,0,POINT (-2797500.000 -2417500.000)


In [8]:
sum(my_data["SEC"].isna())

720313

In [9]:
ais_basins = gpd.read_file("aux_files/IMBIE_AIS_Basins/ANT_Basins_IMBIE2_v1.6.shp")
ais_basins = ais_basins.reset_index().rename(columns={"index": "basin_id"}).to_crs("epsg:4326")
ais_basins["basin_id"] = ais_basins["basin_id"].astype(str)
ais_basins.head()

,basin_id,Regions,Subregion,geometry
0,0,Islands,None,"MULTIPOLYGON (((108.91949 -66.91047, 108.91933..."
1,1,West,H-Hp,"POLYGON ((-90.24588 -74.02074, -90.53000 -73.9..."
2,2,West,F-G,"POLYGON ((-144.36800 -75.62323, -144.36771 -75..."
3,3,East,E-Ep,"POLYGON ((-151.52291 -85.37243, -151.30170 -85..."
4,4,East,D-Dp,"POLYGON ((144.62895 -67.18320, 144.62074 -67.1..."


In [10]:
sf_type = 1
marker_size = 1

In [11]:
# crs_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
# transformed = my_data.to_crs(crs_3031)

# fig, ax = plt.subplots(
#     figsize=(8, 6), facecolor="white", subplot_kw=dict(projection=crs_3031)
# )  # Create our plot

# transformed[transformed["SEC"].notna()].plot(
#     column="SEC",
#     ax=ax,
#     legend=True,
#     vmax=2,
#     vmin=-2,
#     marker="s",
#     markersize=marker_size,
#     cmap="bwr_r",
# )

# ais_basins.plot(color="none", edgecolor="black", ax=ax, alpha=0.5, lw=0.7)

# # ax.coastlines(resolution="50m", color="black")  # Add coastlines
# gl = ax.gridlines(draw_labels=True, color="black", alpha=0.25)
# gl.ylabel_style = {
#     "color": "black",
#     "alpha": 0.5,
# }
# ax.set_xlim(-2.8e6, 2.8e6)
# ax.set_ylim(-2.8e6, 2.8e6)

# fig.show()

In [12]:
timedata_df = pd.read_csv(
    "/home/jgnq4/Documents/eocis/landice_dash_test/processed_files/time_series_data_3.csv"
).sort_values("code")

max_v = np.round(timedata_df["SEC"].abs().max(axis=None) + 0.05, 1)
timedata_df = timedata_df.reset_index(drop=True).reset_index()
timedata_df["basin"] = timedata_df["basin"].astype(str)
timedata_df = pd.merge(
    timedata_df,
    ais_basins[["basin_id", "Subregion"]],
    how="outer",
    left_on="basin",
    right_on="basin_id",
).drop(columns="basin_id")
timedata_df

,index,code,period,midpoint,basin,SEC,Subregion
0,0,199107-199607,1991/07 - 1996/07,1994.0759,0,-0.153539,None
1,36,199108-199608,1991/08 - 1996/08,1994.1608,0,-0.164086,None
2,55,199109-199609,1991/09 - 1996/09,1994.2456,0,-0.164086,None
3,71,199110-199610,1991/10 - 1996/10,1994.3276,0,-0.164086,None
4,98,199111-199611,1991/11 - 1996/11,1994.4125,0,-0.164086,None
...,...,...,...,...,...,...,...
6535,6449,201805-202305,2018/05 - 2023/05,2020.9082,all,-0.004625,NaN
6536,6469,201806-202306,2018/06 - 2023/06,2020.9932,all,-0.004625,NaN
6537,6483,201807-202307,2018/07 - 2023/07,2021.0753,all,-0.004625,NaN
6538,6503,201808-202308,2018/08 - 2023/08,2021.1603,all,-0.004625,NaN


In [16]:
import altair as alt
import vl_convert as vlc
import json
from IPython.display import Image

alt.data_transformers.enable("vegafusion")

click_state = alt.selection_point(fields=["Subregion"])

map = (
    alt.Chart(
        ais_basins,
    )
    .mark_geoshape()
    .encode(
        color=alt.Color("Subregion:N").legend(None),
        opacity=alt.condition(click_state, alt.value(1), alt.value(0.2)),
        tooltip=["basin_id:N", "Subregion:N", "Regions:N"],
    )
    .project(type="stereographic")
    .properties(width=300, height=300)
)

jchart1 = alt.JupyterChart(map.add_params(click_state))
jchart1

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_2_store'}…

In [17]:
# nearest = alt.selection_point(nearest=True, on="mouseover", fields=["index"], empty=False)

line = (
    alt.Chart(timedata_df, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X("midpoint:Q", axis=alt.Axis(labels=False), title="Time Period"),
        alt.Y(
            "SEC:Q",
            scale=alt.Scale(domain=(-max_v, max_v)),
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_state, alt.value(1), alt.value(0.05)),
        color=alt.Color("basin:N").legend(None),
        tooltip=["SEC", "period", "Subregion"],
    )
    .properties(width=900, height=300)
)

# # Draw a rule at the location of the selection
# rules = base.mark_rule(color="gray").encode(
#     x="index:Q",
#     opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
#     tooltip=[
#         alt.Tooltip("period", type="nominal", title="Time period"),
#     ],
# )

# points = line.mark_circle().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero)
    .mark_rule(color="black", strokeDash=[1])
    .encode(y="zero:Q", size=alt.value(1))
)

line_chart = line + zero_rule

jchart2 = alt.JupyterChart(line_chart.add_params(click_state))
jchart2

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_2_store'}…

In [18]:
chart = (map | line_chart).add_params(click_state)


jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_2_store'}…